In [33]:
%cd c:\\Users\\joaol\\repos\\NOTAM_proj\\\\lx-notams
%pwd

c:\Users\joaol\repos\NOTAM_proj\lx-notams


c:\Users\joaol\repos\NOTAM_proj\lx-notams\venv_lx\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'c:\\Users\\joaol\\repos\\NOTAM_proj\\lx-notams'

In [34]:
import os
from pathlib import Path



In [35]:
exports_folder = Path.cwd() / 'lx_notams' / 'datasets' / 'exports'


In [36]:
import pandas as pd

dfs = []

for p in exports_folder.glob('*'):
    
    dfs.append(pd.read_csv(p, sep=';'))

In [37]:
test_text = pd.concat(dfs).iloc[0]['fulltext']


In [38]:
%pwd

'c:\\Users\\joaol\\repos\\NOTAM_proj\\lx-notams'

In [39]:
from lx_notams.models.notam.re_maps import REGEX_FEATURE_MAPS, REGEX_Q_FEATURE_MAPS, REGEX_HEADER_FEATURE_MAPS
from lx_notams.models.notam.db_notam import NotamProcessor

In [40]:
test_text 

'A8017/17 NOTAMR A5539/17 Q)SARR/QFFCG/IV/NBO/A/000/999/2612S05813W005 A)SARF B)1711302342 C)1802282359EST E)FIRE AND RESCUE DOWNGRADED TO CAT 5'

In [41]:
df = pd.concat(dfs).reset_index()

In [42]:
a = [i for 
     i in NotamProcessor._apply_regex_feature_map(test_text, REGEX_FEATURE_MAPS)]

In [43]:
a

[{'nat': 'R'},
 {'header': 'A8017/17 NOTAMR A5539/17'},
 {'lat_deg': 26.0},
 {'lat_min': 12.0},
 {'lat_hem': 'S'},
 {'lon_deg': 58.0},
 {'lon_min': 13.0},
 {'lon_hem': 'W'},
 {'fir': 'SARR'},
 {'subject': 'FF'},
 {'condition': 'CG'},
 {'traffic': 'IV'},
 {'purp': 'NBO'},
 {'scope': 'A'},
 {'flmin': 0},
 {'flmax': 999},
 {'rad': 5.0},
 {'Q': 'SARR/QFFCG/IV/NBO/A/000/999/2612S05813W005'},
 {'A': 'SARF'},
 {'B': '1711302342'},
 {'C': '1802282359EST'},
 {'D': None},
 {'E': 'FIRE AND RESCUE DOWNGRADED TO CAT 5'},
 {'F': None},
 {'G': None}]

In [44]:
from collections import ChainMap

records =  df['fulltext'].apply(lambda row: dict(ChainMap(*[ i for i in NotamProcessor._apply_regex_feature_map(row, REGEX_FEATURE_MAPS)])))

In [45]:
notam_df = pd.DataFrame.from_records(records)

# Reference datasets

In [46]:
from lx_notams.models.notam.re_maps import DATASET_MAP

In [47]:
datasets = {}
for k, v in DATASET_MAP.items():
    try:
        datasets[k] = v['import_fn'](v['path'])
    except Exception as e:
        print(f"Could not import {k}")
    

# Enrich E 
Add subject, condition, E, F, G fields 

In [48]:
records[0]


{'G': None,
 'F': None,
 'E': 'FIRE AND RESCUE DOWNGRADED TO CAT 5',
 'D': None,
 'C': '1802282359EST',
 'B': '1711302342',
 'A': 'SARF',
 'Q': 'SARR/QFFCG/IV/NBO/A/000/999/2612S05813W005',
 'rad': 5.0,
 'flmax': 999,
 'flmin': 0,
 'scope': 'A',
 'purp': 'NBO',
 'traffic': 'IV',
 'condition': 'CG',
 'subject': 'FF',
 'fir': 'SARR',
 'lon_hem': 'W',
 'lon_min': 13.0,
 'lon_deg': 58.0,
 'lat_hem': 'S',
 'lat_min': 12.0,
 'lat_deg': 26.0,
 'header': 'A8017/17 NOTAMR A5539/17',
 'nat': 'R'}

In [49]:
notam_df = notam_df \
    .merge(datasets['subjects'][['code', 'sign']], how='left', left_on='subject', right_on='sign')\
        .drop('code', axis=1)\
            .rename(columns = {'sign':'subject_text'})

notam_df['subject_text'] = notam_df['subject_text'].fillna(' ')


In [50]:
notam_df = notam_df \
.merge(datasets['conditions'][['code', 'sign']], how='left', left_on='condition', right_on='sign')\
    .drop('code', axis=1)\
        .rename(columns = {'sign':'condition_text'})

notam_df['condition_text'] = notam_df['condition_text'].fillna('')


In [51]:
notam_df['E'] = notam_df['E'].fillna('')
notam_df['F'] = notam_df['F'].fillna('')
notam_df['G'] = notam_df['G'].fillna('')

In [52]:
notam_df['enrichedE'] = notam_df.apply(lambda row: ' '.join(
    [e for e in [
        row['subject_text'],
        row['condition_text'],
        row['E'],
        row['F'],
        row['G']

        ] if len(e) > 0]
    ), axis=1)

# Categoricals


In [53]:
notam_df['nat'] = notam_df['nat'].fillna('')
notam_df['nat_N'] = notam_df['nat'].apply(lambda x: 1 if 'N' in x else 0)

In [54]:
notam_df['traffic'] = notam_df['traffic'].fillna('')
notam_df['traffic_ifr'] = notam_df['traffic'].apply(lambda x: 1 if 'I' in x else 0)
notam_df['traffic_vfr'] = notam_df['traffic'].apply(lambda x: 1 if 'V' in x else 0)
notam_df['traffic_checklist'] = notam_df['traffic'].apply(lambda x: 1 if 'K' in x else 0)

In [55]:
notam_df['purp'] = notam_df['purp'].fillna('')
notam_df['purp_N'] = notam_df['purp'].apply(lambda x: 1 if 'N' in x else 0)
notam_df['purp_B'] = notam_df['purp'].apply(lambda x: 1 if 'B' in x else 0)
notam_df['purp_O'] = notam_df['purp'].apply(lambda x: 1 if 'O' in x else 0)
notam_df['purp_M'] = notam_df['purp'].apply(lambda x: 1 if 'M' in x else 0)

In [56]:
notam_df['scope'] = notam_df['scope'].fillna('')
notam_df['scope_A'] = notam_df['scope'].apply(lambda x: 1 if 'A' in x else 0)
notam_df['scope_E'] = notam_df['scope'].apply(lambda x: 1 if 'E' in x else 0)
notam_df['scope_W'] = notam_df['scope'].apply(lambda x: 1 if 'W' in x else 0)

In [68]:
notam_df.iloc[0]

G                                                              
F                                                              
E                           FIRE AND RESCUE DOWNGRADED TO CAT 5
D                                                          None
C                                                 1802282359EST
B                                                    1711302342
A                                                          SARF
Q                    SARR/QFFCG/IV/NBO/A/000/999/2612S05813W005
rad                                                         5.0
flmax                                                     999.0
flmin                                                       0.0
scope                                                         A
purp                                                        NBO
traffic                                                      IV
condition                                                    CG
subject                                 

In [72]:
notam_df.loc[notam_df['lat_hem']=='S', ['lat_sig']] = -1
notam_df.loc[notam_df['lat_hem']=='N', ['lat_sig']] = 1
notam_df['lat'] = notam_df['lat_sig']*(notam_df['lat_deg']+(notam_df['lat_min']/60))

notam_df.loc[notam_df['lon_hem']=='W', ['lon_sig']] = -1
notam_df.loc[notam_df['lon_hem']=='E', ['lon_sig']] = 1
notam_df['lng'] = notam_df['lon_sig']*(notam_df['lon_deg']+(notam_df['lon_min']/60))

# Closest Airport Distance

In [57]:
datasets.keys()

dict_keys(['subjects', 'conditions', 'contractions', 'stopwords', 'ngram_exclusion', 'special_numbers', '4lc_destinations', '4lc_alternates', '4lc_2nd_alternate_com', '4lc_2nd_alternate_fw', '4lc_3rd_alternate_com', '4lc_3rd_alternate_fw', '4lc_4th_alternate_com', '4lc_4th_alternate_fw', '4lc_5th_alternate_com', '4lc_5th_alternate_fw', '4lc_6th_alternate_com', '4lc_6th_alternate_fw', '4lc_7th_alternate_com', '4lc_7th_alternate_fw', 'feature_names', 'key_ngrams', 'airlines_sampledataset', 'airports_sampledataset'])

In [58]:
datasets['4lc_destinations']

['BIKF',
 'BKPR',
 'CYUL',
 'DTTJ',
 'EBBR',
 'EDDF',
 'EDDH',
 'EDDL',
 'EDDM',
 'EDDN',
 'EDDS',
 'EDDT',
 'EDDV',
 'EFKT',
 'EFRO',
 'EGBB',
 'EGCC',
 'EGKK',
 'EGLC',
 'EGLL',
 'EHAM',
 'EIDW',
 'EKCH',
 'ELLX',
 'ENGM',
 'EPWA',
 'ESGG',
 'ESSA',
 'FAOR',
 'GCFV',
 'GCLP',
 'GCRR',
 'GCTS',
 'GMAD',
 'GMMX',
 'HECA',
 'HEGN',
 'HELX',
 'HEMA',
 'HESH',
 'HKJK',
 'HTDA',
 'KBOS',
 'KEWR',
 'KJFK',
 'KLAX',
 'KMIA',
 'KORD',
 'KSFO',
 'LBWN',
 'LCLK',
 'LDSP',
 'LEAL',
 'LEBL',
 'LEIB',
 'LEJR',
 'LEMD',
 'LEMG',
 'LEMH',
 'LEPA',
 'LEVC',
 'LFKJ',
 'LFLL',
 'LFMN',
 'LFPG',
 'LFSB',
 'LGAV',
 'LGIR',
 'LGKO',
 'LGMK',
 'LGRP',
 'LGSA',
 'LGSR',
 'LHBP',
 'LICA',
 'LICC',
 'LIEE',
 'LIEO',
 'LIMC',
 'LIPZ',
 'LIRF',
 'LIRQ',
 'LKPR',
 'LLBG',
 'LOWW',
 'LPFR',
 'LPPR',
 'LPPT',
 'LROP',
 'LSGG',
 'LSZH',
 'LTAI',
 'LTBA',
 'LTBS',
 'LTFE',
 'LWSK',
 'LYBE',
 'OLBA',
 'OMDB',
 'OOMS',
 'RJAA',
 'SBGR',
 'UKBB',
 'ULLI',
 'UUDD',
 'VABB',
 'VHHH',
 'VIDP',
 'VTBS',
 'WSSS',
 'ZBAA',
 

In [75]:
len(notam_df)

5112

In [77]:
from geopy.distance import great_circle
import numpy as np

def shortest_destination(lat, lng, airports_datasets):
    airports = list(set([a for a_d in airports_datasets for a in datasets[a_d]]))
    
    relevant_airports = datasets['airports_sampledataset'].loc[datasets['airports_sampledataset']['4lc'].isin(airports)]

    relevant_airports['dist'] = relevant_airports.apply(lambda row: great_circle((lat, lng),(row['lat'], row['lng'])), axis=1)

    return relevant_airports.loc[relevant_airports['dist']==min(relevant_airports['dist']), ['4lc','dist']].iloc[0]

notam_df[['1', '2']] = notam_df.apply(lambda row: shortest_destination(row['lat'],row['lng'],['4lc_destinations', '4lc_alternates']) if not (np.isnan(row['lat']) or np.isnan(row['lng'])) else np.nan, axis=1)\
    .rename(columns={'4lc':'1', 'dist':'2'})